In [9]:
!pip install opendatasets
!pip install pandas
!pip install tensorflow

  Using cached tensorflow-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (21 kB)
  Using cached tensorflow_io_gcs_filesystem-0.37.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached optree-0.14.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.me

In [2]:
import opendatasets as od 
import pandas 

od.download( 
    "https://www.kaggle.com/datasets/msambare/fer2013/data") 


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  sharifuln38


Your Kaggle Key:

  ········


Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013


In [3]:
import os

# List the files in the downloaded folder
dataset_path = "fer2013"
os.listdir(dataset_path)

['train', 'test']

In [11]:
import torch

# Checking if CUDA is available
flag_cuda = torch.cuda.is_available()

if not flag_cuda:
    print('Using CPU')
else:
    print('Using GPU')


Using GPU


In [45]:

from torchvision import transforms

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure single-channel grayscale
    transforms.Resize((48, 48)),  # Resize to match FER2013 dimensions
    transforms.ToTensor(),  # Convert PIL image to PyTorch tensor ✅
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize pixel values
])

from torch.utils.data import DataLoader
from torchvision import datasets
# Load FER2013 dataset
# Load train and test datasets
train_dataset = datasets.ImageFolder(root="./fer2013/train", transform=transform)
valid_dataset = datasets.ImageFolder(root="./fer2013/test", transform=transform)




In [46]:
# Dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [47]:
# Get dataset sizes
train_N = len(train_dataset)
valid_N = len(valid_dataset)

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout


# Hyperparameters
n_classes = 7  # FER2013 has 7 emotions
IMG_CHS = 1  # Grayscale images
kernel_size = 3
flattened_img_size = 75 * 3 * 3

# Define the CNN Model
model = nn.Sequential(
    # First convolution
    nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1),  # 25 x 48 x 48
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 25 x 24 x 24
    
    # Second convolution
    nn.Conv2d(25, 50, kernel_size, stride=1, padding=1),  # 50 x 24 x 24
    nn.BatchNorm2d(50),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.MaxPool2d(2, stride=2),  # 50 x 12 x 12
    
    # Third convolution
    nn.Conv2d(50, 75, kernel_size, stride=1, padding=1),  # 75 x 12 x 12
    nn.BatchNorm2d(75),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 75 x 6 x 6
    
    # Flatten to Dense
    nn.Flatten(),
    nn.Linear(75 * 6 * 6, 512),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(512, n_classes)  # Output 7 classes
)


In [49]:
device = torch.device("cuda" if flag_cuda else "cpu")
model = model.to(device)


In [50]:
# Define Loss and Optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [51]:
# Function to calculate batch accuracy
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [52]:
# Validation function
def validate():
    loss = 0
    accuracy = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    
    print(f'Valid - Loss: {loss:.4f} Accuracy: {accuracy:.4f}')

In [53]:
# Training function
def train():
    loss = 0
    accuracy = 0
    model.train()
    
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        optimizer.zero_grad()
        
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    
    print(f'Train - Loss: {loss:.4f} Accuracy: {accuracy:.4f}')


In [54]:
# Training loop
epochs = 20

for epoch in range(epochs):
    print(f'Epoch: {epoch+1}/{epochs}')
    train()
    validate()

Epoch: 1/20
Train - Loss: 1437.9203 Accuracy: 0.3693
Valid - Loss: 310.0239 Accuracy: 0.4692
Epoch: 2/20
Train - Loss: 1226.5922 Accuracy: 0.4725
Valid - Loss: 287.1896 Accuracy: 0.5106
Epoch: 3/20
Train - Loss: 1138.3527 Accuracy: 0.5154
Valid - Loss: 281.0195 Accuracy: 0.5192
Epoch: 4/20
Train - Loss: 1072.5078 Accuracy: 0.5419
Valid - Loss: 263.3839 Accuracy: 0.5522
Epoch: 5/20
Train - Loss: 1025.3668 Accuracy: 0.5648
Valid - Loss: 267.7942 Accuracy: 0.5393
Epoch: 6/20
Train - Loss: 978.2644 Accuracy: 0.5843
Valid - Loss: 253.9451 Accuracy: 0.5733
Epoch: 7/20
Train - Loss: 934.9874 Accuracy: 0.6041
Valid - Loss: 251.6647 Accuracy: 0.5747
Epoch: 8/20
Train - Loss: 896.0227 Accuracy: 0.6214
Valid - Loss: 246.1778 Accuracy: 0.5860
Epoch: 9/20
Train - Loss: 858.8292 Accuracy: 0.6375
Valid - Loss: 250.8829 Accuracy: 0.5814
Epoch: 10/20
Train - Loss: 822.6898 Accuracy: 0.6489
Valid - Loss: 256.8684 Accuracy: 0.5750
Epoch: 11/20
Train - Loss: 788.2360 Accuracy: 0.6635
Valid - Loss: 246.264